In [ ]:
import torch
from torch import nn

In [ ]:
class block(nn.Module):
  def __init__(self,inn,inter,idt_ds=None,stride=1):
    super().__init__()
    self.expansion=4
    self.l1=nn.Conv2d(inn,inter,kernel_size=1,stride=1,padding=0,bias=False)
    self.bn1=nn.BatchNorm2d(inter)
    self.l2=nn.Conv2d(inter,inter,kernel_size=3,stride=stride,padding=1,bias=False)
    self.l3=nn.Conv2d(inter,inter*self.expansion,kernel_size=1,stride=1,padding=0,bias=False)
    self.bn2=nn.BatchNorm2d(inter*self.expansion)
    self.relu=nn.ReLU()
    self.idt_ds=idt_ds
    self.stride=stride

  def forward(self,x):
    idt=x.clone()

    x=self.bn2(self.l3(self.relu(self.bn1(self.l2(self.relu(self.bn1(self.l1(x))))))))
    if self.idt_ds is not None:
      idt=self.idt_ds(idt)
    x+=idt
    x=self.relu(x)
    return x

In [ ]:
class ResNet(nn.Module):
  def __init__(self,block,layers,img_in,numc):
    super(ResNet,self).__init__()
    self.inchannels=64
    self.l1=nn.Conv2d(img_in,64,kernel_size=7,stride=2,padding=3,bias=False)
    self.bn1 = nn.BatchNorm2d(64)
    self.relu = nn.ReLU()
    self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

    self.la1=self._make_layer(block,layers[0],inter=64,stride=1)
    self.la2=self._make_layer(block,layers[1],inter=128,stride=2)
    self.la3=self._make_layer(block,layers[2],inter=256,stride=2)
    self.la4=self._make_layer(block,layers[3],inter=512,stride=2)

    self.avgpool=nn.AdaptiveAvgPool2d((1,1))
    self.fc=nn.Linear(512*4,numc)

  def forward(self,x):
      x=self.la4(self.la3(self.la2(self.la1(self.maxpool(self.relu(self.bn1(self.l1)))))))
      x=self.avgpool(x)
      x=torch.flattenn(x,1)
      x=self.fc(x)

      return x

  def _make_layer(self,block,layers,inter,stride):
      idt_ds=None
      layer=[]

      if stride!=1 or self.inn !=inter*4:
        idt_ds=nn.Sequential(
            nn.Conv2d(self.inn,inter*4,kernel_size=1,stride=stride,bias=False),
            nn.BatchNorm2d(inter*4)
        )
      layer.append(block(self.inn,inter,idt_ds,stride)
      )

      self.inn=inter*4

      for i in range(layers-1):
        layer.append(block(self.inn,inter))

      return nn.Sequential(*layer)

  def ResNet50(img_in=3,numc=1000):
      return ResNet(block, [3, 4, 6, 3], img_in, numc)


  def ResNet101(img_in=3, numcs=1000):
      return ResNet(block, [3, 4, 23, 3], img_in, numc)


  def ResNet152(img_in=3, numc=1000):
      return ResNet(block, [3, 8, 36, 3], img_in, numc)
